In [1]:
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
import requests

from tqdm import tqdm

import pandas as pd 
#import numpy as np
#import math
import os

In [2]:
FOLDER_PATH = 'D:/JD_Foreclosure_Reports/'
os.makedirs(FOLDER_PATH,exist_ok=True)

In [3]:
# 用selenium收集房源标题以及报告附件的下载链接
driver = webdriver.Chrome()
driver.implicitly_wait(3)  # the waiting time if cannot find an element

In [10]:
# 打开网页
driver.get("https://pmsearch.jd.com/?publishSource=7&provinceId=1")
sleep(1)
# 勾选限定条件
driver.find_element(By.XPATH,'/html/body/div[4]/div/div/div[2]/div[2]/div/div[2]/div/ul/li[1]/a').click()
sleep(1)
driver.find_element(By.XPATH,'/html/body/div[4]/div/div/div[2]/div[3]/div/div[2]/div[1]/ul/li[2]').click()
sleep(1)

In [11]:
%%time
url_list = []
n_page = 1
while True:
    entry_list = driver.find_elements(By.CLASS_NAME,'goods-container')
    print('page {}: find {} entries.'.format(n_page,len(entry_list)))
    for entry in entry_list:
        url = entry.get_attribute('href')
        #print(url)
        url_list.append(url)
        
    # to next page
    next_page = driver.find_elements(By.CLASS_NAME,'ui-pager-next')
    if next_page:
        n_page += 1
        next_page[0].click()  # 必须保持浏览器页面正常大小，否则报错，因为是按照位置来点击的，模拟人的鼠标操作
        sleep(1)
    else:
        break

page 1: find 40 entries.
page 2: find 40 entries.
page 3: find 40 entries.
page 4: find 40 entries.
page 5: find 40 entries.
page 6: find 40 entries.
page 7: find 40 entries.
page 8: find 40 entries.
page 9: find 40 entries.
page 10: find 40 entries.
page 11: find 40 entries.
page 12: find 40 entries.
page 13: find 40 entries.
page 14: find 40 entries.
page 15: find 40 entries.
page 16: find 40 entries.
page 17: find 40 entries.
page 18: find 40 entries.
page 19: find 40 entries.
page 20: find 40 entries.
page 21: find 40 entries.
page 22: find 40 entries.
page 23: find 40 entries.
page 24: find 40 entries.
page 25: find 40 entries.
page 26: find 40 entries.
page 27: find 40 entries.
page 28: find 40 entries.
page 29: find 40 entries.
page 30: find 40 entries.
page 31: find 40 entries.
page 32: find 40 entries.
page 33: find 40 entries.
page 34: find 40 entries.
page 35: find 40 entries.
page 36: find 40 entries.
page 37: find 40 entries.
page 38: find 40 entries.
page 39: find 40 entr

In [13]:
print("{} entries in total.".format(len(url_list)))

9632 entries in total.


In [19]:
url_list[3534]

'https://paimai.jd.com/109858667'

In [16]:
import time
timestr = time.strftime("%Y%m%d")
file_path = os.path.join(FOLDER_PATH,'entry_url_list{}.csv'.format(timestr))
pd.Series(url_list).to_csv(file_path)

In [4]:
url_list = pd.read_csv('D:/JD_Foreclosure_Reports/entry_url_list20220617.csv')

In [5]:
url_list = url_list.iloc[:,1]

In [6]:
url_list = url_list.to_list()

In [7]:
len(url_list)

9632

In [25]:
url_list[4]

'https://paimai.jd.com/288412613'

In [13]:
9632-6538

3094

In [8]:
# at very start, run this cell.
i = 8820

In [92]:
url_list.index('https://paimai.jd.com/279790677')

5932

In [21]:
# to skip the current entry, run this cell.
i += 1

In [22]:
# to check the current entry, run this cell.
i

8967

In [23]:
# 用requests下载文件到指定文件夹
# 网页上的文件名(report_name)自跟文件类型后缀，所以不用加

# if exception happens, run this cell again.
# to skip the current entry, run the above cell and then this cell

for url in tqdm(url_list[i:]):
#for url in ['https://paimai.jd.com/287456648']:
    driver.get(url)
    title = driver.find_elements(By.CLASS_NAME,'pm-name')  # use ctrl+F to check if the class name is unique
    if title:
        title = title[0].text
        #print(title)
        #attachFiles = driver.find_elements(By.CLASS_NAME,'attachFiles')
        #if attachFiles:
        #    attachFiles = attachFiles[0]
        files = driver.find_elements(By.CLASS_NAME,'file')
        if files:
            print(title,url)
            for f in files:
                report = f.find_element(By.XPATH,'a[1]')
                report_name = report.text
                report_url = report.get_attribute('href')
                print(report_name,report_url)
                entry_path = os.path.join(FOLDER_PATH,title)
                os.makedirs(entry_path,exist_ok=True)
                download_path = os.path.join(entry_path,report_name)

                r = requests.get(report_url)
                with open(download_path, 'wb') as file:
                    file.write(r.content)
    i += 1

  0%|          | 0/665 [00:00<?, ?it/s]

【一拍】北京市朝阳区利泽西街6号院8号楼9层1单元1001室，及车位 https://paimai.jd.com/107036508
执行依据.docx https://storage.jd.com/auction.gateway/ebe2a94e34a66b607021524cb067a61120180919165142697.docx
评估报告2.docx https://storage.jd.com/auction.gateway/82346b823750caa291c25e861c29c55920180919165142714.docx
评估报告1.docx https://storage.jd.com/auction.gateway/92c4226c13cca6cf7afbc4d039e3112320180919165142725.docx


  0%|          | 1/665 [00:07<1:17:56,  7.04s/it]

【一拍】北京市丰台区芳城园一区16号楼8层1单元805号房地产（138.39㎡） https://paimai.jd.com/107016156
执行裁定.docx https://storage.jd.com/auction.gateway/0343de3a5abe7e242e71fee3ff78fef320180918095750517.docx
北京市丰台区芳城园一区16号楼8层1单元805号报告.pdf https://storage.jd.com/auction.gateway/d1212b04a2ddcf64b4371cbc683d5c5f20180918095750565.pdf


  0%|          | 2/665 [00:10<57:02,  5.16s/it]  

...

 89%|████████▊ | 590/665 [35:30<05:47,  4.64s/it]

【一拍】北京市朝阳区拂林园3号楼1001号 https://paimai.jd.com/103503450
拂林园估价技术报告 - 副本.pdf https://storage.jd.com/auction.gateway/8ad0da15ed14fb40ea3b3d78491fa75220170726202807520.pdf


 89%|████████▉ | 591/665 [35:35<05:53,  4.78s/it]

【一拍】北京市海淀区太月园8号楼2210号 https://paimai.jd.com/103503214
结果报告（太月园）.doc https://storage.jd.com/auction.gateway/5b0fdefdb258a3ca001f2a2f1a0321eb20170726192831485.doc


100%|██████████| 665/665 [42:52<00:00,  3.87s/it]
